In [1]:
from faker import Faker
import pandas as pd
import random
from datetime import datetime

# DATA EXTRACTION

In [2]:
# Initialize Faker
fake = Faker()
# Set seed for reproducibility
SEED = 42
random.seed(SEED)
fake.seed_instance(SEED)

# Parameters
num_rows = 1000
num_customers = 100
num_products = 50  # unique products
countries_list = [fake.country() for _ in range(random.randint(5, 10))]

# Create unique customer IDs
customer_ids = [str(random.randint(10000, 99999)) for _ in range(num_customers)]

# Product categories
categories = [
    "Electronics", "Clothing", "Home & Kitchen", "Sports", 
    "Office Supplies", "Travel"
]

# List of realistic product names
product_types = [
    "T-Shirt", "Mug", "Notebook", "Backpack", "Pen", "Water Bottle",
    "Cap", "Sunglasses", "Headphones", "Scarf", "Wallet", "Watch",
    "Phone Case", "Laptop Sleeve", "Planner", "Keychain", "Desk Lamp",
    "Coffee Maker", "Travel Pillow", "Yoga Mat", "Sneakers", "Jacket",
    "Earbuds", "Hoodie", "Blanket", "Belt", "Thermos", "Tote Bag",
    "Book", "Candle", "Picture Frame", "Charger", "Power Bank",
    "Mouse Pad", "Desk Organizer", "Clock", "Gloves", "Raincoat",
    "Slippers", "Bowl", "Plate", "Cutlery Set", "Back Scratcher",
    "Umbrella", "Tripod", "Camping Tent", "Flashlight", "Mirror", "Floor Mat", "Calculator"
]

# Ensure exactly num_products unique products
selected_products = random.sample(product_types, num_products)

# Generate product catalog with StockCode ↔ Description ↔ Category
product_catalog = {}
for product_name in selected_products:
    stock_code = str(random.randint(10000, 99999))
    while stock_code in product_catalog:  # Ensure uniqueness
        stock_code = str(random.randint(10000, 99999))
    price = round(random.uniform(1, 100), 2)
    category = random.choice(categories)
    product_catalog[stock_code] = {
        "name": product_name,
        "price": price,
        "category": category
    }

product_codes = list(product_catalog.keys())
product_weights = [random.randint(1, 100) for _ in range(num_products)]
country_weights = [random.randint(1, 100) for _ in countries_list]

# Functions
def generate_invoice_no():
    """Generate a 6-digit invoice number (no cancellations)."""
    return str(random.randint(100000, 999999))

def generate_quantity():
    """Generate mostly positive quantities (2% chance negative)."""
    if random.random() < 0.02:
        return random.randint(-5, -1)
    return random.randint(1, 50)

def generate_invoice_date():
    """Generate random invoice date in last 2 years."""
    start_date = datetime(2023, 8, 12)
    end_date = datetime(2025, 8, 12)
    return fake.date_time_between(start_date=start_date, end_date=end_date)

def get_unit_price(stock_code):
    """Return product's fixed price (1% chance negative for errors)."""
    if random.random() < 0.01:
        return round(random.uniform(-10, -0.5), 2)
    return product_catalog[stock_code]["price"]

# Build dataset ensuring each product appears at least once
records = []

# Step 1: One record per product to guarantee appearance
for stock_code in product_codes:
    records.append({
        "InvoiceNo": generate_invoice_no(),
        "StockCode": stock_code,
        "Description": product_catalog[stock_code]["name"],
        "Category": product_catalog[stock_code]["category"],
        "Quantity": generate_quantity(),
        "InvoiceDate": generate_invoice_date(),
        "UnitPrice": get_unit_price(stock_code),
        "CustomerID": random.choice(customer_ids),
        "Country": random.choices(countries_list, weights=country_weights, k=1)[0]
    })

# Step 2: Generate remaining random records
for _ in range(num_rows - num_products):
    stock_code = random.choices(product_codes, weights=product_weights, k=1)[0]
    records.append({
        "InvoiceNo": generate_invoice_no(),
        "StockCode": stock_code,
        "Description": product_catalog[stock_code]["name"],
        "Category": product_catalog[stock_code]["category"],
        "Quantity": generate_quantity(),
        "InvoiceDate": generate_invoice_date(),
        "UnitPrice": get_unit_price(stock_code),
        "CustomerID": random.choice(customer_ids),
        "Country": random.choices(countries_list, weights=country_weights, k=1)[0]
    })

# Create DataFrame
df = pd.DataFrame(records)


In [3]:
df.describe()

,Quantity,InvoiceDate,UnitPrice
count,1000.00000,1000,1000.000000
mean,26.42700,2024-08-14 00:33:51.394999808,50.683280
min,-5.00000,2023-08-12 07:34:02,-8.500000
25%,14.00000,2024-02-11 19:26:09.500000,17.350000
50%,27.00000,2024-08-09 06:55:58.500000,58.040000
75%,39.00000,2025-02-16 16:58:24.249999872,80.260000
max,50.00000,2025-08-11 09:38:57,97.220000
std,14.79539,NaN,31.983754


In [4]:
df.head()

,InvoiceNo,StockCode,Description,Category,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,800305,69470,Pen,Travel,20,2025-01-18 14:35:33,1.32,13905,Saint Pierre and Miquelon
1,240814,44522,Laptop Sleeve,Clothing,7,2025-03-09 07:02:50,97.22,13478,Saint Vincent and the Grenadines
2,734210,76542,Gloves,Travel,22,2025-07-04 23:25:57,91.35,93227,Burundi
3,629959,49117,Camping Tent,Office Supplies,4,2024-10-08 14:22:08,84.33,91070,Burundi
4,146228,89818,Sneakers,Home & Kitchen,-4,2023-10-18 12:39:26,20.69,46421,Anguilla


In [5]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Category               object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

## checking for rows with negative values for quantities and prices

In [6]:
negative_quantities=df[df['Quantity']<0]
print(len(negative_quantities))
negative_quantities

22


,InvoiceNo,StockCode,Description,Category,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
4,146228,89818,Sneakers,Home & Kitchen,-4,2023-10-18 12:39:26,20.69,46421,Anguilla
70,416950,98039,Slippers,Sports,-1,2023-09-16 14:13:42,65.34,28289,Saint Pierre and Miquelon
112,799233,92719,Calculator,Electronics,-2,2025-01-24 18:08:04,6.83,70589,Ecuador
138,238205,44760,Hoodie,Sports,-4,2024-12-23 07:37:38,92.73,46048,Anguilla
199,745239,92719,Calculator,Electronics,-5,2025-06-22 07:04:55,6.83,87236,Burundi
202,662093,14117,Mouse Pad,Electronics,-1,2024-09-03 19:36:05,86.10,38785,Anguilla
274,139286,30289,Yoga Mat,Clothing,-1,2024-05-28 01:03:45,19.80,55082,Saint Vincent and the Grenadines
320,901467,22899,Water Bottle,Travel,-4,2024-06-23 04:08:51,7.00,10851,Bouvet Island (Bouvetoya)
400,566686,77391,Mirror,Travel,-3,2024-08-24 00:12:45,24.56,91070,Czech Republic
418,835575,42591,Back Scratcher,Office Supplies,-2,2025-06-04 07:51:03,19.97,59823,Ecuador


In [7]:
negative_prices=df[df['UnitPrice']<0]
print(len(negative_prices))
negative_prices

10


,InvoiceNo,StockCode,Description,Category,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
42,902784,47388,Picture Frame,Travel,31,2024-06-02 02:21:29,-6.66,14207,Falkland Islands (Malvinas)
78,282256,50306,Belt,Clothing,11,2025-06-15 00:25:34,-5.72,44993,Falkland Islands (Malvinas)
279,247442,50306,Belt,Clothing,16,2023-11-13 23:40:54,-6.36,35203,Burundi
322,606676,56438,Book,Sports,2,2023-10-24 06:21:37,-4.76,13278,Northern Mariana Islands
512,753898,77391,Mirror,Travel,7,2024-10-14 16:50:10,-0.51,80284,Ecuador
530,656829,46509,Power Bank,Electronics,44,2025-05-30 04:30:59,-3.03,46463,Czech Republic
605,508422,36365,Desk Organizer,Sports,35,2024-04-26 00:23:09,-3.16,81482,Burundi
662,137812,50306,Belt,Clothing,42,2024-03-05 04:39:06,-8.50,38221,Burundi
807,563339,92719,Calculator,Electronics,39,2023-12-26 14:45:55,-1.62,38221,Czech Republic
919,116389,22833,Plate,Office Supplies,6,2024-01-29 10:18:56,-8.30,65302,Czech Republic


# DATA TRANSFORMATION

### Removing rows with negative Quantity or UnitPrice

In [8]:
df_cleaned = df[(df["Quantity"] >= 0) & (df["UnitPrice"] > 0)]

print(f"Rows before cleaning: {len(df)}")
print(f"Rows after cleaning: {len(df_cleaned)}")

Rows before cleaning: 1000
Rows after cleaning: 968


### ADDING THE CALCULATED COLUMN TOTAL SALES

In [9]:
df_cleaned["TotalSales"] = df_cleaned['Quantity'] * df_cleaned['UnitPrice']
# round the TotalSales to 2 decimal places
df_cleaned["TotalSales"] = df_cleaned["TotalSales"].round(2)
df_cleaned.head(2)

C:\Users\Admin\AppData\Local\Temp\ipykernel_31864\1678482026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["TotalSales"] = df_cleaned['Quantity'] * df_cleaned['UnitPrice']
C:\Users\Admin\AppData\Local\Temp\ipykernel_31864\1678482026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["TotalSales"] = df_cleaned["TotalSales"].round(2)


,InvoiceNo,StockCode,Description,Category,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales
0,800305,69470,Pen,Travel,20,2025-01-18 14:35:33,1.32,13905,Saint Pierre and Miquelon,26.40
1,240814,44522,Laptop Sleeve,Clothing,7,2025-03-09 07:02:50,97.22,13478,Saint Vincent and the Grenadines,680.54


### TOTAL REVENUE GENERATED PER COUNTRY

In [10]:
total_revenue_per_country = df_cleaned.groupby('Country')['TotalSales'].sum().reset_index()
total_revenue_per_country = total_revenue_per_country.sort_values(by='TotalSales', ascending=False)
total_revenue_per_country.head()

,Country,TotalSales
4,Czech Republic,217043.30
6,Falkland Islands (Malvinas),214272.24
3,Burundi,192193.78
9,Saint Vincent and the Grenadines,175347.90
0,Anguilla,166530.65


### TOTAL SALES PER PRODUCT

In [11]:
total_revenue_per_Description = df_cleaned.groupby('Description')['TotalSales'].sum().reset_index()
total_revenue_per_Description = total_revenue_per_Description.sort_values(by='TotalSales', ascending=False)
total_revenue_per_Description.head()

,Description,TotalSales
19,Gloves,105509.25
35,Raincoat,94506.75
21,Hoodie,92822.73
17,Flashlight,75170.26
8,Candle,73999.75


### TOTAL SALES PER CUSTOMER

In [12]:
total_sales_per_customer = df_cleaned.groupby('CustomerID')['TotalSales'].sum().reset_index()
total_sales_per_customer = total_sales_per_customer.sort_values(by='TotalSales', ascending=False)
total_sales_per_customer.head()

,CustomerID,TotalSales
93,94939,28550.27
96,97841,27808.98
82,85674,27179.00
66,62581,26771.87
95,96673,26211.90


### FILTERING THE DATA SET TO GET THE RECORDS OF LAST YEAR

In [13]:
last_year = datetime.now().year - 1
df_last_year = df_cleaned[df_cleaned['InvoiceDate'].dt.year == last_year]
df_last_year.head()

,InvoiceNo,StockCode,Description,Category,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales
3,629959,49117,Camping Tent,Office Supplies,4,2024-10-08 14:22:08,84.33,91070,Burundi,337.32
5,563298,31174,Floor Mat,Office Supplies,28,2024-11-12 15:34:41,54.40,22280,Bouvet Island (Bouvetoya),1523.20
6,824586,10074,Cutlery Set,Sports,35,2024-07-04 17:11:34,60.30,80284,Czech Republic,2110.50
10,938742,73699,Scarf,Office Supplies,27,2024-01-28 19:42:26,81.79,17331,Saint Pierre and Miquelon,2208.33
11,920391,26483,Desk Lamp,Sports,43,2024-02-08 17:06:48,13.71,65392,Saint Vincent and the Grenadines,589.53


# LOAD

In [14]:
import sqlite3

In [15]:
# Connect to SQLite
conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# Create Customer Dimension
customer_dim = (
    df_cleaned[["CustomerID", "Country"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
customer_dim["CustomerKey"] = customer_dim.index + 1  # surrogate key

# Create Time Dimension
time_dim = pd.DataFrame()
time_dim["InvoiceDate"] = df_cleaned["InvoiceDate"].drop_duplicates().reset_index(drop=True)
time_dim["DateKey"] = time_dim.index + 1
time_dim["Year"] = time_dim["InvoiceDate"].dt.year
time_dim["Month"] = time_dim["InvoiceDate"].dt.month
time_dim["Day"] = time_dim["InvoiceDate"].dt.day
time_dim["Weekday"] = time_dim["InvoiceDate"].dt.day_name()
time_dim["Quarter"] = time_dim["InvoiceDate"].dt.quarter  # for roll-up queries

# Create Product Dimension (with category)
product_dim = (
    df_cleaned[["StockCode", "Description", "UnitPrice", "Category"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
product_dim["ProductKey"] = product_dim.index + 1

# Mapping Keys to Fact Table
df_fact = (
    df_cleaned
    .merge(customer_dim, on=["CustomerID", "Country"], how="left")
    .merge(time_dim, on="InvoiceDate", how="left")
    .merge(product_dim, on=["StockCode", "Description", "UnitPrice", "Category"], how="left")
)

# Fact Table (only keep numeric keys + measures)
sales_fact = df_fact[[
    "InvoiceNo",
    "CustomerKey",
    "DateKey",
    "ProductKey",
    "Quantity",
    "UnitPrice"
]]

# Logs for loaded data
print(f"CustomerDim rows: {len(customer_dim)}")
print(f"TimeDim rows: {len(time_dim)}")
print(f"ProductDim rows: {len(product_dim)}")
print(f"SalesFact rows: {len(sales_fact)}")

# Load into SQLite
customer_dim.to_sql("CustomerDim", conn, if_exists="replace", index=False)
time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)
product_dim.to_sql("ProductDim", conn, if_exists="replace", index=False)
sales_fact.to_sql("SalesFact", conn, if_exists="replace", index=False)

# Commit & Close
conn.commit()
conn.close()

print("✅ Data successfully loaded into retail_dw.db")


CustomerDim rows: 571
TimeDim rows: 968
ProductDim rows: 50
SalesFact rows: 968
✅ Data successfully loaded into retail_dw.db


In [16]:

#Connecting to the database
conn = sqlite3.connect("retail_dw.db")

# Checking table names 
print("📂 Tables in database:")
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

# Preview each dimension table 
print("\n👥 CustomerDim (first 5 rows):")
print(pd.read_sql("SELECT * FROM CustomerDim LIMIT 5;", conn))   

print("\n📅 TimeDim (first 5 rows):")
print(pd.read_sql("SELECT * FROM TimeDim LIMIT 5;", conn))

print("\n📦 ProductDim (first 5 rows):")
print(pd.read_sql("SELECT * FROM ProductDim LIMIT 5;", conn))

print("\n📊 SalesFact (first 5 rows):")
print(pd.read_sql("SELECT * FROM SalesFact LIMIT 5;", conn))

conn.close()


📂 Tables in database:
          name
0  CustomerDim
1      TimeDim
2   ProductDim
3    SalesFact

👥 CustomerDim (first 5 rows):
  CustomerID                           Country  CustomerKey
0      13905         Saint Pierre and Miquelon            1
1      13478  Saint Vincent and the Grenadines            2
2      93227                           Burundi            3
3      91070                           Burundi            4
4      22280         Bouvet Island (Bouvetoya)            5

📅 TimeDim (first 5 rows):
           InvoiceDate  DateKey  Year  Month  Day   Weekday  Quarter
0  2025-01-18T14:35:33        1  2025      1   18  Saturday        1
1  2025-03-09T07:02:50        2  2025      3    9    Sunday        1
2  2025-07-04T23:25:57        3  2025      7    4    Friday        3
3  2024-10-08T14:22:08        4  2024     10    8   Tuesday        4
4  2024-11-12T15:34:41        5  2024     11   12   Tuesday        4

📦 ProductDim (first 5 rows):
  StockCode    Description  UnitPrice    

### Saving the cleaned_df to a CSV file

In [17]:
df_cleaned.to_csv("data\cleaned_retail_data.csv", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Admin\AppData\Local\Temp\ipykernel_31864\2945209726.py:1: SyntaxWarning: invalid escape sequence '\c'
  df_cleaned.to_csv("data\cleaned_retail_data.csv", index=False)
